In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from torchvision.datasets import Food101
from pathlib import Path
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [2]:
MODEL_DIRECTORY = 'models'
PATH = Path(MODEL_DIRECTORY)
PATH.mkdir(parents= True, exist_ok= True)

MODEL_NAME = 'PyTorch_NaiveBayes.pth'
MODEL_PATH = PATH / MODEL_NAME

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224
    transforms.ToTensor(),  # Convert the images to PyTorch tensors
    transforms.Lambda(lambda x: x.flatten())  # Flatten the images to a 1D array
])

In [4]:
train_dataset = Food101('data', split= 'train', transform= transform, target_transform= None, download= True)
test_dataset = Food101('data', split= 'test', transform= transform, target_transform= None, download= True)

In [14]:
def convert_to_numpy(loader):
    data_list = []
    label_list = []
    print(len(loader))
    for data, labels in tqdm(loader, total=len(loader)):
        print(data.shape)
        data_list.append(data.numpy())
        label_list.append(labels.numpy())
    data_array = np.concatenate(data_list)
    label_array = np.concatenate(label_list)
    return data_array, label_array

In [6]:
# Load the Food101 dataset into PyTorch DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:
# Convert DataLoader to numpy array
X_train, y_train = convert_to_numpy(train_loader)
X_test, y_test = convert_to_numpy(val_loader)

2368
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])
torch.Size([32, 12288])


KeyboardInterrupt: 

In [ ]:
# Create a Gaussian Naive Bayes classifier object
model = GaussianNB()

# Fit the classifier on the training data
model.fit(X_train, y_train)

In [ ]:
# Predict the classes for the validation data
y_pred = model.predict(X_test)

In [ ]:
# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))